# Pipeline for independent providers' lower limb metric data

In [ ]:
#Installing required packages

! pip install pyodbc
! pip install openpyxl
! pip install pandas

### Creating date and datetime variables for use in queries and output filenames

In [ ]:
# Import date class from datetime module
from datetime import datetime

# create variable for current date and time, rounded to nearest second (for filename of Excel extract at end)
now = datetime.now()

now_ns = (now.strftime("%Y-%m-%d-%H-%M-%S"))
print(now_ns)

# create variable for start of reporting month and reporting month, in yyyy-mm[-dd] format 
# (start of reporting month is for SQL queries, reporting month is for filename of Excel extract)
start_of_month = datetime(2025, 7, 1)
start_of_month_d = start_of_month.strftime("%Y-%m-%d")
reporting_month = start_of_month.strftime("%Y-%m")
print(start_of_month_d)
print(reporting_month)

# create variable for CSDS month ID (1499 = February 2025, 1500 = March 2025, etc. This needs to be incremented each reporting month)
csds_month_id = 1504

### Running SQL query for DQ metrics

In [ ]:
# Query to connect to SQL Server and run a SQL query for DQ metrics (PU5001-2)

import env
import pyodbc
import pandas as pd
import os


def data_connection_dq():
    """
    Using the user's credentials, retrieve the data from the UDAL Warehouse

    returns:
    pandas dataframe of the data
    """

    server = env.SERVER
    database = env.DATABASE
    uid = env.UID

    conn_str = (
        f'DRIVER={{ODBC Driver 17 for SQL Server}};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={uid};'
        'Authentication=ActiveDirectoryInteractive;'
    )

    conn = pyodbc.connect(conn_str)


#    table = os.getenv('table')
#   subtable = os.getenv('subtable')

    query = f"""
    DECLARE @MonthStart1 as DATE DECLARE @MonthEnd1 as DATE 
    SET @MonthStart1 = '{start_of_month_d}'
    SET @MonthEnd1 = EOMONTH(@MonthStart1)

    SELECT
	@MonthEnd1 AS [ReportingDate],
	[OrgID_Provider] as [ProviderCode],
	[CodedFinding] as [CodedFindingOrProcedure],
	count(*) as [NumberOfRecords]

	from [Reporting_MESH_CSDS].[CYP202CareActivity]
    
    where [OrgID_Provider] in ('NQA', 'NR5', 'NAY')
    and [Unique_MonthID] = '{csds_month_id}'

	group by
	[OrgID_Provider],
	[CodedFinding]
    """
    
    query2 = f"""
    DECLARE @MonthStart1 as DATE DECLARE @MonthEnd1 as DATE 
    SET @MonthStart1 = '{start_of_month_d}' 
    SET @MonthEnd1 = EOMONTH(@MonthStart1)

    SELECT
	@MonthEnd1 AS [ReportingDate],
	[OrgID_Provider] as [ProviderCode],
	[CodedProcedure] as [CodedFindingOrProcedure],
	count(*) as [NumberOfRecords]

	from [Reporting_MESH_CSDS].[CYP202CareActivity]
    
    where [OrgID_Provider] in ('NQA', 'NR5', 'NAY')
    and [Unique_MonthID] = '{csds_month_id}'

	group by
	[OrgID_Provider],
	[CodedProcedure]
    """

    # Execute the query and load it into a pandas DataFrame
    df = pd.read_sql(query, conn)
    df2 = pd.read_sql(query2, conn)
    
    # Close the connection after retrieving data
    conn.close()

    df["Metric_ID"] = 'PU5001'
    df2["Metric_ID"] = 'PU5002'

    df_combined = pd.concat([df, df2])

    return df_combined

In [ ]:
# Running function created above to generate the SQL DQ query results as one merged and transformed dataframe
df = data_connection_dq()

# Creating copy of dataframe to work with in case of errors
df_copy = df

# Checking the generated dataframe looks right by inspecting first 10 rows
df_copy.head(10)

### Transforming DQ metric data into summary format

In [ ]:
#Function to transform dataframe into table to email to IPs

import pandas as pd

def transform_ll_dq_data(input_df):

    # Step 1: Define input data
    df = input_df

    # Step 2: Prepare numeratorValue and denominatorValue for each ProviderCode
    # numeratorValue: count of non-null values for each ProviderCode
    numerator = df[df['CodedFindingOrProcedure'].notna()].groupby(['ProviderCode', 'Metric_ID'])['NumberOfRecords'].sum().reset_index(name='numeratorValue')

    # denominatorValue: count of all rows (including nulls) for each ProviderCode
    denominator = df.groupby(['ProviderCode', 'Metric_ID'])['NumberOfRecords'].sum().reset_index(name='denominatorValue')

    # Merge numerator and denominator counts
    counts = pd.merge(numerator, denominator, on=['ProviderCode', 'Metric_ID'])

    # Step 3: Calculate value = (numeratorValue / denominatorValue) * 100
    counts['value'] = (counts['numeratorValue'] / counts['denominatorValue']) * 100

    # Step 4: Get rangeMin and rangeMax from the 'value' column
    # rangeMin = counts['value'].min()
    # rangeMax = counts['value'].max()

    # Step 5: Since the reportingDate might vary, let's associate each ProviderCode with its reportingDate(s).
    # Assuming the ReportingDate is consistent per ProviderCode, take the first ReportingDate for each ProviderCode.
    reporting_dates = df.groupby('ProviderCode')['ReportingDate'].first().reset_index()

    # Step 6: Build the final DataFrame with the specified columns and order
    df_transformed = counts.merge(reporting_dates, on='ProviderCode')

    df_transformed = df_transformed.rename(columns={
        'ProviderCode': 'providerCode',
        'ReportingDate': 'reportingDate',
        'Metric_ID': 'metricID'
    })

    # Reorder columns as requested
    df_transformed = df_transformed[
        ['metricID', 'providerCode', 'reportingDate', 'value', 
        'numeratorValue', 'denominatorValue']
    ]
    
    return df_transformed

In [ ]:
# Run function above on DQ dataframe
df_copy_transformed = transform_ll_dq_data(df_copy)

# Checking the transformed dataframe looks right by inspecting first 10 rows
df_copy_transformed.head(10)

### Running SQL query for main lower limb metric data

In [ ]:
# Query to connect to SQL Server and run a SQL query for the LL wound code metrics (PU5003-7)

import env
import pyodbc
import pandas as pd
import os


def data_connection_ll_codes():
    """
    Using the user's credentials, retrieve the data from the UDAL Warehouse

    returns:
    pandas dataframe of the data
    """

    server = env.SERVER
    database = env.DATABASE
    uid = env.UID

    conn_str = (
        f'DRIVER={{ODBC Driver 17 for SQL Server}};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={uid};'
        'Authentication=ActiveDirectoryInteractive;'
    )

    conn = pyodbc.connect(conn_str)


#    table = os.getenv('table')
#   subtable = os.getenv('subtable')

    query = f"""
        
    DECLARE @MonthStart1 as DATE
    DECLARE @MonthEnd1 as DATE
    SET @MonthStart1 = '{start_of_month_d}'
    SET @MonthEnd1 = EOMONTH(@MonthStart1)

    SELECT
    @MonthEnd1 AS [ReportingDate],
    Contacts.OrgID_Provider AS [ProviderCode],
    COUNT (DISTINCT Contacts.Person_ID) AS Unique_Patients,

    CASE WHEN Activity.CodedFinding IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7','413167008','XaJWE','Xa6nc',
    '298006005','XaQn5','8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','Y36af','Y36b0','Ua1De','X77cy','X77cz','X79mk','P5sb.','p5vW.',
    'p5vX.','p5z9.','p5zA.','Xa6nb')
    or Activity.CodedProcedure IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7','413167008','XaJWE','Xa6nc',
    '298006005','XaQn5','8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','Y36af','Y36b0','Ua1De','X77cy','X77cz','X79mk','P5sb.','p5vW.',
    'p5vX.','p5z9.','p5zA.','Xa6nb') THEN 'LegUlcer_Code_Present'
    ELSE 'Other' END AS Metric


    FROM [Reporting_MESH_CSDS].[CYP201CareContact] AS [Contacts]

    LEFT JOIN [Reporting_MESH_CSDS].[CYP202CareActivity] AS [Activity]
    ON Contacts.Person_ID = Activity.Person_ID
    AND Contacts.UniqueSubmissionID = Activity.UniqueSubmissionID
    AND Contacts.Unique_CareContactID = Activity.Unique_CareContactID
    AND Contacts.AuditID = Activity.AuditID

    LEFT JOIN [Reporting_MESH_CSDS].[DerLatestFlag] AS [Submissions]
    ON Contacts.UniqueSubmissionID = Submissions.UniqueSubmissionID
    AND Contacts.OrgID_Provider = Submissions.OrgID_Provider
    AND Submissions.RP_EndDate = @MonthEnd1

    WHERE 
    Contacts.Contact_Date BETWEEN @MonthStart1 AND @MonthEnd1
    AND Submissions.DerIsLatest = '1'
    AND (CASE WHEN AttendanceStatus IS NULL THEN AttendOrNot ELSE AttendanceStatus END IN ('5','6'))
    and Contacts.OrgID_Provider in ('NQA', 'NR5', 'NAY')

    GROUP BY
    Contacts.OrgID_Provider,
    CASE WHEN Activity.CodedFinding IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7','413167008','XaJWE','Xa6nc',
    '298006005','XaQn5','8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','Y36af','Y36b0','Ua1De','X77cy','X77cz','X79mk','P5sb.','p5vW.',
    'p5vX.','p5z9.','p5zA.','Xa6nb')
    or Activity.CodedProcedure IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7','413167008','XaJWE','Xa6nc',
    '298006005','XaQn5','8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','Y36af','Y36b0','Ua1De','X77cy','X77cz','X79mk','P5sb.','p5vW.',
    'p5vX.','p5z9.','p5zA.','Xa6nb') THEN 'LegUlcer_Code_Present'
    ELSE 'Other' END

    union

    SELECT
    @MonthEnd1 AS [ReportingDate],
    Contacts.OrgID_Provider AS [ProviderCode],
    COUNT (DISTINCT Contacts.Person_ID) AS Unique_Patients,

    CASE WHEN Activity.CodedFinding IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7') 
    or Activity.CodedProcedure IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7') THEN 'LegUlcer_Present'
    WHEN Activity.CodedProcedure IN ('8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','X77cy','X77cz','X79mk') 
    or Activity.CodedFinding IN ('8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','X77cy','X77cz','X79mk') THEN 'LegUlcer_Assessment'
    WHEN Activity.CodedProcedure IN ('Y36af','Y36b0','Ua1De','P5sb.','p5vW.','p5vX.','p5z9.','p5zA.') or Activity.CodedFinding IN ('Y36af','Y36b0','Ua1De','P5sb.',
    'p5vW.','p5vX.','p5z9.','p5zA.') THEN 'LegUlcer_Treatment'
    WHEN Activity.CodedFinding IN ('413167008','XaJWE') or Activity.CodedProcedure IN ('413167008','XaJWE') THEN 'LegUlcer_TreatmentNotSpecified'
    WHEN Activity.CodedFinding IN ('Xa6nc','298006005','XaQn5','Xa6nb') or Activity.CodedProcedure IN ('Xa6nc','298006005','XaQn5','Xa6nb') THEN 'LegUlcer_Healed'
    ELSE 'Other' END AS Metric


    FROM [Reporting_MESH_CSDS].[CYP201CareContact] AS [Contacts]

    LEFT JOIN [Reporting_MESH_CSDS].[CYP202CareActivity] AS [Activity]
    ON Contacts.Person_ID = Activity.Person_ID
    AND Contacts.UniqueSubmissionID = Activity.UniqueSubmissionID
    AND Contacts.Unique_CareContactID = Activity.Unique_CareContactID
    AND Contacts.AuditID = Activity.AuditID

    LEFT JOIN [Reporting_MESH_CSDS].[DerLatestFlag] AS [Submissions]
    ON Contacts.UniqueSubmissionID = Submissions.UniqueSubmissionID
    AND Contacts.OrgID_Provider = Submissions.OrgID_Provider
    AND Submissions.RP_EndDate = @MonthEnd1

    WHERE 
    Contacts.Contact_Date BETWEEN @MonthStart1 AND @MonthEnd1
    AND Submissions.DerIsLatest = '1'
    AND (CASE WHEN AttendanceStatus IS NULL THEN AttendOrNot ELSE AttendanceStatus END IN ('5','6'))
    and Contacts.OrgID_Provider in ('NQA', 'NR5', 'NAY')
    
    GROUP BY
    Contacts.OrgID_Provider,
    CASE WHEN Activity.CodedFinding IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7') 
    or Activity.CodedProcedure IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7') THEN 'LegUlcer_Present'
    WHEN Activity.CodedProcedure IN ('8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','X77cy','X77cz','X79mk') 
    or Activity.CodedFinding IN ('8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','X77cy','X77cz','X79mk') THEN 'LegUlcer_Assessment'
    WHEN Activity.CodedProcedure IN ('Y36af','Y36b0','Ua1De','P5sb.','p5vW.','p5vX.','p5z9.','p5zA.') or Activity.CodedFinding IN ('Y36af','Y36b0','Ua1De','P5sb.',
    'p5vW.','p5vX.','p5z9.','p5zA.') THEN 'LegUlcer_Treatment'
    WHEN Activity.CodedFinding IN ('413167008','XaJWE') or Activity.CodedProcedure IN ('413167008','XaJWE') THEN 'LegUlcer_TreatmentNotSpecified'
    WHEN Activity.CodedFinding IN ('Xa6nc','298006005','XaQn5','Xa6nb') or Activity.CodedProcedure IN ('Xa6nc','298006005','XaQn5','Xa6nb') THEN 'LegUlcer_Healed'
    ELSE 'Other' END
    """
    
    # Execute the query and load it into a pandas DataFrame
    df = pd.read_sql(query, conn)
    
    # Close the connection after retrieving data
    conn.close()

    return df

In [ ]:
# Running function created above to generate the SQL DQ query results as one merged and transformed dataframe
df2 = data_connection_ll_codes()

# Creating copy of dataframe to work with in case of errors
df2_copy = df2

# Checking the generated dataframe looks right by inspecting first 10 rows
df2_copy.head(10)

### Transforming main metric data into summary format

In [ ]:
def transform_ll_metric_data(input_df):

    # Mapping for recoding the Metric column including the new entry
    metric_map = {
        'LegUlcer_Code_Present': 'PU5003',
        'LegUlcer_Present': 'PU5004',
        'LegUlcer_Assessment': 'PU5005',
        'LegUlcer_Treatment': 'PU5006',
        'LegUlcer_Healed': 'PU5007'
    }

    # Read the Excel file into a DataFrame
    df = input_df

    # Recode Metric column and drop rows not in mapping
    df['metricID'] = df['Metric'].map(metric_map)
    df = df.dropna(subset=['metricID'])

    # Rename and reorder columns, also transfer data accordingly
    df_transformed = pd.DataFrame({
        'metricID': df['metricID'],
        'providerCode': df['ProviderCode'],
        'reportingDate': df['ReportingDate'],
        'value': df['Unique_Patients'],
        'numeratorValue': '',  # blank column
        'denominatorValue': '',# blank column
    })

    # Convert 'value' column to numeric if not already (just in case)
    df_transformed['value'] = pd.to_numeric(df_transformed['value'], errors='coerce')
 
    return df_transformed


In [ ]:
# Run function above on DQ dataframe
df2_copy_transformed = transform_ll_metric_data(df2_copy)

# Checking the transformed dataframe looks right by inspecting first 10 rows
df2_copy_transformed.head(10)

### Combining the two metric dataframes

In [ ]:
# Concatenating the two dataframes
import pandas as pd

df_all_metrics = pd.concat([df_copy_transformed, df2_copy_transformed])

In [ ]:
df_all_metrics.head(10)

### Adding metric descriptions column

In [ ]:
# function to add metric descriptions column

def add_metric_desc(input_df):
    
    mapping = {
        'PU5001': 'Percentage completeness of the Clinical Coded Finding field in the CSDS',
        'PU5002': 'Percentage completeness of the Clinical Coded Procedure field in the CSDS',
        'PU5003': 'Number of unique patients on the community service caseload who have a lower limb code',
        'PU5004': 'Number of unique patients on the community service caseload who have a lower limb code - Presence',
        'PU5005': 'Number of unique patients on the community service caseload who have a lower limb code - Assessment',
        'PU5006': 'Number of unique patients on the community service caseload who have a lower limb code - Treatment',
        'PU5007': 'Number of unique patients on the community service caseload who have a lower limb code - Healed'
    }

    df = input_df

    df['metricDesc'] = df['metricID'].map(mapping)

    return df

In [ ]:
df_all_metrics_desc = df_all_metrics

add_metric_desc(df_all_metrics_desc)

df_all_metrics_desc.head(10)

### Creating filtered dataframes for each provider's extract

In [ ]:
#function to filter dataframe by ODS code

def filter_by_prov(input_df, ods_code):
    
    df = input_df
    
    output_df = df[df['providerCode'] == ods_code]
    
    return output_df

In [ ]:
df_all_metrics_desc_nqa = filter_by_prov(df_all_metrics_desc, 'NQA')

df_all_metrics_desc_nqa.head(10)

In [ ]:
df_all_metrics_desc_nr5 = filter_by_prov(df_all_metrics_desc, 'NR5')

df_all_metrics_desc_nr5.head(10)

In [ ]:
"""
df_all_metrics_desc_nay = df_all_metrics_desc

filter_by_prov(df_all_metrics_desc_nqa, 'NAY')

df_all_metrics_desc_nay.head(10)
"""

In [ ]:
# Function to export dataframe to Excel

def export_to_excel(df, output_file):

    df.to_excel(output_file, sheet_name='Insert', index=False)

    print(f"Transformation complete. Data saved to {output_file}")
    

### Exporting filtered dataframes to Excel

In [ ]:
export_to_excel(df_all_metrics_desc_nqa, f'C:/Users/owen.kenn (DDCC7590)/NHS/Quality and Improvement Analytics - Productivity/Wound Care/Wound Care - LL 2025/Excel outputs/nqa_all_metrics_{reporting_month}_{now_ns}.xlsx')

In [ ]:
export_to_excel(df_all_metrics_desc_nr5, f'C:/Users/owen.kenn (DDCC7590)/NHS/Quality and Improvement Analytics - Productivity/Wound Care/Wound Care - LL 2025/Excel outputs/nr5_all_metrics_{reporting_month}_{now_ns}.xlsx')

In [ ]:
# export_to_excel(df_all_metrics_desc_nay, f'C:/Users/owen.kenn (DDCC7590)/NHS/Quality and Improvement Analytics - Productivity/Wound Care/Wound Care - LL 2025/Excel outputs/nay_all_metrics_{reporting_month}_{now_ns}.xlsx')